# Agents

- LlamaIndex supports 3 main types of reasoning agents:
  1. Function Calling Agents: These work with AI models that can call specific functions.
  2. ReAct Agents: These can work with any AI that does chat or text endpoints and deal with complex reasoning tasks.
  3. Advanced Custom Agents: These use more compplex methods to deal with more complex tasks and workflows.


In [1]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.tools import FunctionTool
def multiply(a:int,b:int)->int:
    '''Multiply two integers and returns the resulting integer'''
    return a*b


In [2]:
llm=HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
agent=AgentWorkflow.from_tools_or_functions(
    [FunctionTool.from_defaults(multiply)],
    llm=llm
)

- Agents are by default stateles


In [3]:
response=await agent.run("What is 2 times 2?")
from llama_index.core.workflow import Context
ctx= Context(agent)
response=await agent.run("My nname is Bob.",ctx=ctx)
response=await agent.run("What was my name again?",ctx=ctx)


WorkflowRuntimeError: Error in step 'run_agent_step': 402, message='Payment Required', url='https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions'

In [4]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
db=chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection=db.get_or_create_collection("alfred")
vector_store=ChromaVectorStore(chroma_collection=chroma_collection)
pipeline=IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=25,chunk_overlap=0),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ],
    vector_store=vector_store,
)

In [5]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index=VectorStoreIndex.from_vector_store(vector_store,embed_model=embed_model)
from llama_index.core.tools import QueryEngineTool
query_engine=index.as_query_engine(llm=llm,similarity_top_k=3)


In [6]:
query_engine_tool=QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="name",
    description="a specific description",
    return_direct=False,
)
query_engine_agent=AgentWorkflow.from_tools_or_functions(
    [query_engine_tool],
    llm=llm,
    system_prompt="You are a helpful assistant that has access to a database containing persona descriptions."
)

# Multi-agent systems

- by giving each agent a name and description, the system maintains a single active speaker, with each agent having the ability to hand off to another agent.


In [7]:
from llama_index.core.agent.workflow import (
    AgentWorkflow,
    FunctionAgent,
    ReActAgent,
)

# Define some tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b


# Create agent configs
# NOTE: we can use FunctionAgent or ReActAgent here.
# FunctionAgent works for LLMs with a function calling API.
# ReActAgent works for any LLM.
calculator_agent = ReActAgent(
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="You are a calculator assistant. Use your tools for any math operation.",
    tools=[add, subtract],
    llm=llm,
)

query_agent = ReActAgent(
    name="info_lookup",
    description="Looks up information about XYZ",
    system_prompt="Use your tool to query a RAG system to answer information about XYZ",
    tools=[query_engine_tool],
    llm=llm
)

# Create and run the workflow
agent = AgentWorkflow(
    agents=[calculator_agent, query_agent], root_agent="calculator"
)

# Run the system
response = await agent.run(user_msg="Can you add 5 and 3?")

WorkflowRuntimeError: Error in step 'run_agent_step': 402, message='Payment Required', url='https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions'